In [1]:
! pip install kaggle

In [2]:
!pip install tensorflow

In [3]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data Collection - Kaggle API

In [4]:
kaggle_dictionary = json.load(open('kaggle.json'))
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [7]:
#unzip the dataset file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [8]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


Loading the Dataset

In [9]:
data = pd.read_csv("IMDB Dataset.csv")

In [10]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
data.shape

(50000, 2)

In [12]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [13]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [14]:
data.isnull().sum()

,0
review,0
sentiment,0


In [15]:
data.replace({"sentiment":{"positive":1, "negative":0}}, inplace=True)

In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [17]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [18]:
# split data into training data and test data

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Pre-Processing

In [19]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000) #tokenizer for convert word to numbers
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [20]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [21]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [22]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [23]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


LSTM - Long Short-Term Memory

In [25]:
 # Build the model

 model = Sequential()
 model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
 model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
 model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
# complie the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Training the model

In [30]:
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 129s 258ms/step - accuracy: 0.8229 - loss: 0.4032 - val_accuracy: 0.8661 - val_loss: 0.3242
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 258ms/step - accuracy: 0.8721 - loss: 0.3139 - val_accuracy: 0.8796 - val_loss: 0.3003
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 140s 254ms/step - accuracy: 0.8836 - loss: 0.2858 - val_accuracy: 0.8114 - val_loss: 0.5933
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 128s 255ms/step - accuracy: 0.8734 - loss: 0.3111 - val_accuracy: 0.8631 - val_loss: 0.3296
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 146s 263ms/step - accuracy: 0.9095 - loss: 0.2317 - val_accuracy: 0.8769 - val_loss: 0.3066


In [31]:
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.8786 - loss: 0.3089
Test Loss: 0.3078857958316803
Test Accuracy: 0.8776999711990356


Building a Predictive System

In [33]:
def predict_sentiment(review):
    # tokenize and pad the review
    sequence = tokenizer.texts_to_sequences(review)
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "positive" if prediction > 0.5 else "negative"
    return sentiment

In [34]:
#example usage
new_review = ["This movie was amazing! I loved every minute of it."]
sentiment = predict_sentiment(new_review)
print("Sentiment:", sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
Sentiment: positive


In [35]:
#example usage
new_review = ["This movie was terrible! I hated every minute of it."]
sentiment = predict_sentiment(new_review)
print("Sentiment:", sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Sentiment: negative
